In [ ]:
from keybert import KeyBERT
import whisper_timestamped as whisper
from nltk import download
from nltk.corpus import stopwords

download("stopwords")
YT_LINK = "/content/vid test.weba"

# Hardcoded text

In [ ]:
with open("content/oleg.txt", "r", encoding="utf-8") as f:
    txt = f.read()

# NLP

In [ ]:
from transformers import BertModel, BertTokenizerFast
from datasets import Dataset
import torch
import torch.nn.functional as F
import pandas as pd
from transformers.pipelines import pipeline

In [ ]:
russian_stopwords = set(stopwords.words("russian"))

In [ ]:
model_checkpoint = 'bert-base-multilingual-cased'
hf_model = pipeline("feature-extraction", model="bert-base-multilingual-cased")

tokenizer = BertTokenizerFast.from_pretrained(model_checkpoint)
model = BertModel.from_pretrained(model_checkpoint)
model = model.eval()


def get_embeddings(text):
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)
    with torch.no_grad():
        outputs = model(**inputs)

    embeddings = outputs.pooler_output
    normalized_embeddings = F.normalize(embeddings, p=2)

    return normalized_embeddings

In [ ]:
from pymystem3 import mystem


def preprocess_text(text):
    m = mystem.Mystem()
    lemmas = m.lemmatize(text)
    lemmas = [l for l in lemmas if l.isalpha()]
    return " ".join(lemmas)

In [ ]:
kw_model = KeyBERT(model=hf_model)
window_size = 200
stride = 50

txt = preprocess_text(txt)
txt = txt.replace("\n", " ").replace("\r", " ")
txt = txt.replace(".", "").replace(",", "")
txt = txt.lower()
tokens = txt.split()
tokens = [t for t in tokens if t not in russian_stopwords]

indexed_keywords = {}
frags = []
df = pd.DataFrame(columns=["index", "keywords", "fragments"])
for i in range(0, len(tokens) - window_size, stride):
    frag = tokens[i: i + window_size]
    frag = " ".join([f.lower() for f in frag])
    frags.append(frag)
    keywords = kw_model.extract_keywords(frag, keyphrase_ngram_range=(1, 1), top_n=10, use_mmr=True)
    indexed_keywords[i] = keywords

In [ ]:
values_dict = {"index": [], "keywords": []}
for i, keywords in indexed_keywords.items():
    words = " ".join([k[0] for k in keywords])

    values_dict["index"].append(i)
    values_dict["keywords"].append(words)

In [ ]:
df = pd.DataFrame(values_dict)
df["fragments"] = frags
df.head(3)

In [ ]:
import faiss
import numpy as np

# Retrieve embeddings from the DataFrame
TARGET_COL = 'fragments'

df['embedding'] = df[TARGET_COL].apply(lambda x: get_embeddings(x).numpy()[0])
embeddings = df['embedding'].tolist()

dimension = len(embeddings[0])
index = faiss.IndexFlatL2(dimension)
index.add(np.array(embeddings).astype('float32'))

# Calculate the distance between every row and the next one
distances = []
for i in range(len(embeddings) - 1):
    distance = index.search(np.array([embeddings[i]]).astype('float32'), 2)[0][0][1]
    distances.append(distance)

# Append the distances to the DataFrame
df['distance_to_next'] = distances + [None]  # The last row


In [ ]:
df.head(50)

In [ ]:
df["distance_to_next"].mean()

In [ ]:
words_dataset = Dataset.from_pandas(df)
words_dataset = words_dataset.map(
    lambda x: {'text_embeddings': get_embeddings(x["keywords"])[0]}
)

In [ ]:
words_dataset.to_pandas().head()

In [ ]:
scores, samples = words_dataset.get_nearest_examples('text_embeddings', test_frag_1, k=words_dataset.shape[0])
demo_df = pd.DataFrame({"index": samples['index']})
demo_df["score_to_1"] = scores
demo_df["keywords"] = demo_df["index"].apply(lambda x: [w[0] for w in indexed_keywords[x]])

In [ ]:
df

In [ ]:
demo_df["score/dist"] = demo_df["score_to_1"] / demo_df["index"]

In [36]:
demo_df

,index,score_to_1,keywords,score/dist
0,50,0.000000,"[обучаться, практиковать, собеседник, приглаша...",0.000000
1,1600,0.041696,"[заинтересовывать, встречаться, заинтересованн...",0.000026
2,1650,0.041696,"[заинтересовывать, встречаться, заинтересованн...",0.000025
3,100,0.044245,"[практиковать, взаимодействовать, обучаться, р...",0.000442
4,1550,0.046017,"[встречаться, заинтересовывать, увлекать, заин...",0.000030
5,1500,0.051990,"[поработать, заинтересованность, увлекать, инт...",0.000035
6,1400,0.053479,"[интердисциплинарный, поработать, сформировыва...",0.000038
7,200,0.054806,"[монокорпоративный, ведущая, преподаватель, пр...",0.000274
8,900,0.054976,"[являться, заинтересовывать, пользователь, пос...",0.000061
9,1050,0.055510,"[заинтересовывать, проговаривать, встречаться,...",0.000053


## Sentence transformers

In [ ]:
with open("content/oleg.txt", "r", encoding="utf-8") as f:
    txt = f.read()

In [ ]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("paraphrase-multilingual-MiniLM-L12-v2")

sentences = txt.split(".")
embeddings = model.encode(sentences)

In [ ]:
def get_sentence_embeddings(sentence):
    embeddings = model.encode(sentence)
    return embeddings

In [ ]:
values_dict = {"index":[], "sentence": [], "embedding": []}
for i, sentence in enumerate(sentences):
    values_dict["index"].append(i)
    values_dict["sentence"].append(sentence.strip())
    values_dict["embedding"].append(get_sentence_embeddings(sentence))


In [ ]:
df = pd.DataFrame(values_dict)
df.head(5)

In [ ]:
sentence_dataset = Dataset.from_pandas(df)
sentence_dataset = sentence_dataset.add_faiss_index(column='embedding')

In [ ]:
test_frag_1 = sentence_dataset.to_pandas().head()['embedding'][1]
sentence_dataset.to_pandas().head()

In [ ]:
scores, samples = sentence_dataset.get_nearest_examples('embedding', test_frag_1, k=len(sentence_dataset))
demo_df = pd.DataFrame({"index": samples['index'], "fragment": values_dict["sentence"]})
demo_df["score_to_1"] = scores / scores.mean()

In [ ]:
demo_df

## Sentence transformers

In [ ]:
with open("content/test_text.txt", "r", encoding="utf-8") as f:
    txt = f.read()

In [ ]:
from sklearn.cluster import KMeans, HDBSCAN
from sentence_transformers import SentenceTransformer

In [ ]:
embedder = SentenceTransformer("all-MiniLM-L6-v2")

sentences = txt.split(".")
sentences = [s.strip() for s in sentences if len(s) > 0]
embeddings = model.encode(sentences)

In [ ]:
num_clusters = 4
clustering_model = HDBSCAN(min_cluster_size=num_clusters, max_cluster_size=4)
clustering_model.fit(embeddings)
cluster_assignment = clustering_model.labels_

In [ ]:
clustered_sentences = [[] for i in range(num_clusters)]
for sentence_id, cluster_id in enumerate(cluster_assignment):
    clustered_sentences[cluster_id].append(sentences[sentence_id])


In [ ]:
values_dict = {"index":[], "sentence": []}
for i, sentence in enumerate(sentences):
    values_dict["index"].append(i)
    values_dict["sentence"].append(sentence.strip())

df = pd.DataFrame(values_dict)

In [ ]:
for i, cluster in enumerate(clustered_sentences):
    df[f"cluster_{i}"] = df["sentence"].apply(lambda x: 1 if x in cluster else 0)


In [ ]:
df